In [1]:
import cv2
import torch
import lpips
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
from nnutils.eval_utils import compute_lpips
from scripts.visualize.lpips_models import PerceptualLoss

# WARNING: to import the PerceptualLoss from this notebook without any import errors, change the following lines in the following scripts:
# 1) scripts/visualize/lpips_models/__init__.py
# "from lpips_models import dist_model" -> "from scripts.visualize.lpips_models import dist_model"
# 
# 2) scripts/visualize/lpips_models/dist_model.py
# "import lpips_models as util" -> "import scripts.visualize.lpips_models as util"
#
# 3) scripts/visualize/lpips_models/networks_basic.py
# "import lpips_models as util" -> "import scripts.visualize.lpips_models as util"

In [2]:
# load example images from original LPIPS repo (https://github.com/richzhang/PerceptualSimilarity/tree/master/imgs)
# DOWNLOAD the example images from the link above and place them in Total-Recon/lpips_test_imgs/

ex_p0 = cv2.imread("./lpips_test_imgs/ex_p0.png")[:,:,::-1] / 255.         # H, W, 3, range [0, 1]
ex_p1 = cv2.imread("./lpips_test_imgs/ex_p1.png")[:,:,::-1] / 255.         # H, W, 3, range [0, 1]
ex_ref = cv2.imread("./lpips_test_imgs/ex_ref.png")[:,:,::-1] / 255.       # H, W, 3, range [0, 1]

print("range of example image ex_p0: ", ex_p0.min(), ex_p0.max())
print("range of example image ex_p1: ", ex_p1.min(), ex_p1.max())
print("range of example image ex_ref: ", ex_ref.min(), ex_ref.max())

range of example image ex_p0:  0.00784313725490196 0.996078431372549
range of example image ex_p1:  0.0 1.0
range of example image ex_ref:  0.0 1.0


In [4]:
# 1) Total-Recon's LPIPS implementation: "compute_lpips" (https://github.com/andrewsonga/Total-Recon/blob/7b49ce1a9da0f1a10a79e787b14b7e5c747e5c44/nnutils/eval_utils.py#L111-L132)
# assumes input images as np.ndarrays in the range of [0, 1]
# internally scales the input images to the range of [-1, 1] before feeding it to lpips function

lpips_model = PerceptualLoss(model='net-lin', net='alex', use_gpu=True, version=0.1)
lpips_p0_totalrecon = compute_lpips(ex_p0, ex_ref, lpips_model)
lpips_p1_totalrecon = compute_lpips(ex_p1, ex_ref, lpips_model)

Setting up Perceptual loss...
Loading model from: /home/ndsong/Total-Recon/scripts/visualize/lpips_models/weights/v0.1/alex.pth
...[net-lin [alex]] initialized
...Done


In [5]:
# 2) lpips library's LPIPS implementation: "lpips.LPIPS" (https://pypi.org/project/lpips/)
# assumes input images in the range of [-1, 1]

ex_p0 = torch.from_numpy(ex_p0)[None].float().permute(0, 3, 1, 2) * 2 - 1            # tensor of shape (3, H, W), range [-1, 1]
ex_p1 = torch.from_numpy(ex_p1)[None].float().permute(0, 3, 1, 2) * 2 - 1            # tensor of shape (3, H, W), range [-1, 1]
ex_ref = torch.from_numpy(ex_ref)[None].float().permute(0, 3, 1, 2) * 2 - 1          # tensor of shape (3, H, W), range [-1, 1]

loss_fn_alex = lpips.LPIPS(net='alex') # best forward scores
lpips_p0_lpips = loss_fn_alex(ex_p0, ex_ref)
lpips_p1_lpips = loss_fn_alex(ex_p1, ex_ref)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/ndsong/anaconda3/envs/lpips_test/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth


/home/ndsong/anaconda3/envs/lpips_test/lib/python3.10/site-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_p

In [6]:
# 3) torchmetrics' LPIPS implementation: "torchmetrics.image.lpip.LearnedPerceptualImagePatchSimilarity" (https://torchmetrics.readthedocs.io/en/latest/pages/image_metrics.html#learnedperceptualimagepatchsimilarity)

lpips_torchmetrics = LearnedPerceptualImagePatchSimilarity(net_type='alex')
lpips_p0_lpips = lpips_torchmetrics(ex_p0, ex_ref)
lpips_p1_lpips = lpips_torchmetrics(ex_p1, ex_ref)

/home/ndsong/anaconda3/envs/lpips_test/lib/python3.10/site-packages/torchmetrics/functional/image/lpips.py:325: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state

In [7]:
# Comparing the results between Total-Recon's, lpips library's, and torchmetrics' LPIPS implementations
# lpips_p0
print("LPIPS between ex_p0 and ex_ref:")
print("Total-Recon's LPIPS implementation: ", lpips_p0_totalrecon)
print("lpips library's LPIPS implementation: ", lpips_p0_lpips)
print("torchmetrics' LPIPS implementation: ", lpips_p0_lpips)

LPIPS between ex_p0 and ex_ref:
Total-Recon's LPIPS implementation:  0.719045877456665
lpips library's LPIPS implementation:  tensor(0.7219)
torchmetrics' LPIPS implementation:  tensor(0.7219)


In [8]:
# Comparing the results between Total-Recon's, lpips library's, and torchmetrics' LPIPS implementations
# lpips_p1
print("LPIPS between ex_p1 and ex_ref:")
print("Total-Recon's LPIPS implementation: ", lpips_p1_totalrecon)
print("lpips library's LPIPS implementation: ", lpips_p1_lpips)
print("torchmetrics' LPIPS implementation: ", lpips_p1_lpips)

LPIPS between ex_p1 and ex_ref:
Total-Recon's LPIPS implementation:  0.13566210865974426
lpips library's LPIPS implementation:  tensor(0.1376)
torchmetrics' LPIPS implementation:  tensor(0.1376)


The slight difference in metrics between Total-Recon's implementation and the latest implementations of LPIPS by Zhang et. al and torchmetrics is due to the following:

Total-Recon's lpips implementation was taken from the NSFF repo, which in turn was taken from the original LPIPS library written by Zhang et. al (according to this github issue: https://github.com/zhengqili/Neural-Scene-Flow-Fields/issues/6). The NSFF repo took the version of the original LPIPS library before this commit on September 5th, 2020 (https://github.com/richzhang/PerceptualSimilarity/commit/c33f89e9f46522a584cf41d8880eb0afa982708b), which carried out a large refactoring of the codebase. It is highly likely that the changes made during this commit and all subsequent commits are the reason for the discrepancy shown above. 